<h1 align=center><font size = 5>Start a new business in London</font></h1>

## 1. Introduction

#### London is an international city with a large population and great business potential. If someone wants to start a new business in London, which business to start and who are the competitors are interesting problems for a businessman.

## 2. Data, Resources&Methodology

#### We can use Foursquare to explore the venues in London. Then we can use groupby to check how many each venue is and find out which is the most popular business. In the end, we can use clustering to find out the locations of the most popular business.

### Import necessary Libraries

In [27]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'XA13KGDBJLAXVA05WGA4KAXV1IL530E1JN1SGHBXIKFT2TSC' # your Foursquare ID
CLIENT_SECRET = '0ZBIDP1CETXCT2QVHGPTPUPRAALSBN2CMDSCVWNN5JEWEIO1' # your Foursquare Secret
VERSION = '20200306'
LIMIT = 500
radius=500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XA13KGDBJLAXVA05WGA4KAXV1IL530E1JN1SGHBXIKFT2TSC
CLIENT_SECRET:0ZBIDP1CETXCT2QVHGPTPUPRAALSBN2CMDSCVWNN5JEWEIO1


In [8]:
address = 'London'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5073219 -0.1276474


In [9]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XA13KGDBJLAXVA05WGA4KAXV1IL530E1JN1SGHBXIKFT2TSC&client_secret=0ZBIDP1CETXCT2QVHGPTPUPRAALSBN2CMDSCVWNN5JEWEIO1&ll=51.5073219,-0.1276474&v=20200306&radius=500&limit=500'

In [10]:
results = requests.get(url).json()
'There are {} around London.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around London.'

In [11]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ac518cdf964a520e6a520e3',
  'name': 'National Gallery',
  'location': {'address': 'Trafalgar Sq',
   'lat': 51.50887601013219,
   'lng': -0.1284778118133545,
   'labeledLatLngs': [{'label': 'display',
     'lat': 51.50887601013219,
     'lng': -0.1284778118133545}],
   'distance': 182,
   'postalCode': 'WC2N 5DN',
   'cc': 'GB',
   'city': 'London',
   'state': 'Greater London',
   'country': 'United Kingdom',
   'formattedAddress': ['Trafalgar Sq',
    'London',
    'Greater London',
    'WC2N 5DN',
    'United Kingdom']},
  'categories': [{'id': '4bf58dd8d48988d18f941735',
    'name': 'Art Museum',
    'pluralName': 'Art Museums',
    'shortName': 'Art Museum',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 

#### Tranform the data into a *pandas* dataframe

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.tail(10)

,name,categories,lat,lng
90,The Royal Trafalgar by Thistle,Hotel,51.509087,-0.130328
91,Amorino,Ice Cream Shop,51.511510,-0.125794
92,The Rooftop,Cocktail Bar,51.507335,-0.129216
93,The Comedy Store,Comedy Club,51.510234,-0.132507
94,The Espresso Room,Coffee Shop,51.511074,-0.126568
95,wagamama,Asian Restaurant,51.510792,-0.124721
96,Smith & Wollensky,Steakhouse,51.509557,-0.122241
97,The Travellers Club,Lounge,51.506654,-0.132939
98,Koshari Street,African Restaurant,51.510889,-0.127005
99,M&M's World,Candy Store,51.510599,-0.131149


In [17]:
dataframe_filtered.groupby('categories').count()

,name,lat,lng
categories,,,
African Restaurant,1,1,1
Art Gallery,3,3,3
Art Museum,2,2,2
Asian Restaurant,2,2,2
Bakery,1,1,1
Bar,2,2,2
Bookstore,1,1,1
Boutique,1,1,1
Burger Joint,1,1,1


In [18]:
print('There are {} uniques categories.'.format(len(dataframe_filtered['categories'].unique())))

There are 55 uniques categories.


### Analyze Each name

In [19]:
# one hot encoding
dataframe_onehot = pd.get_dummies(dataframe_filtered[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dataframe_onehot['name'] = dataframe_filtered['name'] 

# move neighborhood column to the first columndataframe_filtered
fixed_columns = [dataframe_onehot.columns[-1]] + list(dataframe_onehot.columns[:-1])
dataframe_onehot = dataframe_onehot[fixed_columns]

dataframe_onehot.head()

,name,African Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bar,Bookstore,Boutique,Burger Joint,...,Seafood Restaurant,Spa,Spanish Restaurant,Speakeasy,Steakhouse,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Wine Bar
0,National Gallery,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Trafalgar Square,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Trafalgar Square Fountain,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Sainsbury Wing National Gallery,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Trafalgar Square Lions,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
dataframe_onehot.shape

(100, 56)

In [21]:
dataframe_grouped = dataframe_onehot.groupby('name').mean().reset_index()
dataframe_grouped

,name,African Restaurant,Art Gallery,Art Museum,Asian Restaurant,Bakery,Bar,Bookstore,Boutique,Burger Joint,...,Seafood Restaurant,Spa,Spanish Restaurant,Speakeasy,Steakhouse,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Wine Bar
0,Adelphi Theatre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Admiralty Arch,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Afternoon Tea at the Corinthia Hotel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Amorino,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bancone,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Barrafina,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,Blame Gloria,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Brumus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Café In The Crypt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Citadines Trafalgar Square London,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
dataframe_grouped.shape

(99, 56)

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['name'] = dataframe_grouped['name']

for ind in np.arange(dataframe_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dataframe_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,name,1st Most Common Venue
0,Adelphi Theatre,Theater
1,Admiralty Arch,Monument / Landmark
2,Afternoon Tea at the Corinthia Hotel,Tea Room
3,Amorino,Ice Cream Shop
4,Bancone,Italian Restaurant


### Cluster Neighborhoods

In [28]:
# set number of clusters
kclusters = 4

dataframe_grouped_clustering = dataframe_grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dataframe_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dataframe_merged = dataframe_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dataframe_merged = dataframe_merged.join(neighborhoods_venues_sorted.set_index('name'), on='name')

dataframe_merged.head() # check the last columns!

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue
0,National Gallery,Art Museum,51.508876,-0.128478,0,Art Museum
1,Trafalgar Square,Plaza,51.507987,-0.128048,0,Plaza
2,East Trafalgar Square Fountain,Fountain,51.508088,-0.127700,0,Fountain
3,Sainsbury Wing National Gallery,Art Museum,51.508384,-0.129001,0,Art Museum
4,Trafalgar Square Lions,Outdoor Sculpture,51.507641,-0.127888,0,Outdoor Sculpture


## 3. Results

In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dataframe_merged['lat'], dataframe_merged['lng'], dataframe_merged['name'], dataframe_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [32]:
dataframe_merged.loc[dataframe_merged['Cluster Labels'] == 0]

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue
0,National Gallery,Art Museum,51.508876,-0.128478,0,Art Museum
1,Trafalgar Square,Plaza,51.507987,-0.128048,0,Plaza
2,East Trafalgar Square Fountain,Fountain,51.508088,-0.127700,0,Fountain
3,Sainsbury Wing National Gallery,Art Museum,51.508384,-0.129001,0,Art Museum
4,Trafalgar Square Lions,Outdoor Sculpture,51.507641,-0.127888,0,Outdoor Sculpture
5,ESPA Life at Corinthia,Spa,51.506402,-0.125114,0,Spa
6,Nelson's Column,Monument / Landmark,51.507744,-0.127931,0,Monument / Landmark
7,St Martin-in-the-Fields,Church,51.508746,-0.126507,0,Church
11,Waterstones,Bookstore,51.507650,-0.127130,0,Bookstore
12,Barrafina,Spanish Restaurant,51.509427,-0.125894,0,Spanish Restaurant


In [33]:
dataframe_merged.loc[dataframe_merged['Cluster Labels'] == 1]

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue
8,Corinthia Hotel,Hotel,51.506607,-0.124460,1,Hotel
18,Haymarket Hotel,Hotel,51.508267,-0.131315,1,Hotel
27,Citadines Trafalgar Square London,Hotel,51.507020,-0.124685,1,Hotel
37,The Royal Horseguards,Hotel,51.506127,-0.124224,1,Hotel
38,Sofitel London St James,Hotel,51.507636,-0.132367,1,Hotel
49,St. Martin's Lane Hotel,Hotel,51.510526,-0.126705,1,Hotel
51,hub by Premier Inn London Covent Garden,Hotel,51.510036,-0.127111,1,Hotel
67,The Z Hotel Piccadilly,Hotel,51.509009,-0.131564,1,Hotel
90,The Royal Trafalgar by Thistle,Hotel,51.509087,-0.130328,1,Hotel


In [34]:
dataframe_merged.loc[dataframe_merged['Cluster Labels'] == 2]

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue
10,Trafalgar Studios,Theater,51.506651,-0.127378,2,Theater
24,The London Coliseum,Theater,51.509846,-0.127005,2,Theater
29,Her Majesty's Theatre,Theater,51.508289,-0.131641,2,Theater
60,Harold Pinter Theatre,Theater,51.509443,-0.131644,2,Theater
64,Prince of Wales Theatre,Theater,51.510132,-0.131594,2,Theater
69,Noel Coward Theatre,Theater,51.511042,-0.127619,2,Theater
82,Adelphi Theatre,Theater,51.510383,-0.123068,2,Theater
88,Wyndham's Theatre,Theater,51.511072,-0.127990,2,Theater


In [35]:
dataframe_merged.loc[dataframe_merged['Cluster Labels'] == 3]

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue
9,National Portrait Gallery,Art Gallery,51.509438,-0.128032,3,Art Gallery
14,Institute of Contemporary Arts (ICA),Art Gallery,51.506265,-0.130817,3,Art Gallery
31,Mall Galleries,Art Gallery,51.506370,-0.129776,3,Art Gallery


## 4. Discussion

#### From the groupby and clustering results, we can see there are 8 theaters and 9 hotels in London, which are outnumbered the other businesses. According to the results, hotel and theater are popular business to invest in. 

#### If a business man wants to open a hotel, the rivals are Corinthia Hotel, Haymarket Hotel, Citadines Trafalgar Square London, The Royal Horseguards, Sofitel London St James, St. Martin's Lane Hotel, hub by Premier Inn London Covent Garden, The Z Hotel Piccadilly and The Royal Trafalgar by Thistle. If someone wants to open a theater, the competitors are Trafalgar Studios, The London Coliseum, Her Majesty's Theatre, Harold Pinter Theatre, 	Prince of Wales Theatre, Noel Coward Theatre, Adelphi Theatre and Wyndham's Theatre	.

## 5. Conclusion

#### In conclusion, this paper used Foursquare to explore the venues in London with the methedology of groupby and clustering to find out that hotel and theater are most popular business in London. For hotel business, the rivals are Corinthia Hotel, Haymarket Hotel, Citadines Trafalgar Square London, The Royal Horseguards, Sofitel London St James, St. Martin's Lane Hotel, hub by Premier Inn London Covent Garden, The Z Hotel Piccadilly and The Royal Trafalgar by Thistle. For theater business, the competitors are Trafalgar Studios, The London Coliseum, Her Majesty's Theatre, Harold Pinter Theatre, Prince of Wales Theatre, Noel Coward Theatre, Adelphi Theatre and Wyndham's Theatre.

#### For further competitor investigation, people can use the competitor results of this research to find out the the competitive advantages and disadvantages before making marketing strategies.